In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

dataset = pd.read_csv('../dataset/train/train.csv')

train_dataset, dev_dataset = train_test_split(dataset, stratify= dataset.label, test_size= 0.1, random_state=1004)

label = train_dataset['label']

print(train_dataset.shape)
print(train_dataset.head(5))

(29180, 6)
          id                                           sentence  \
8902    8906  한편, 지난 25일 자유한국당 이양수 의원은 민주당 강원도당과 충북도당 2곳, 국회...   
30154  30191  2008년 11월 체흐는 스탬퍼드 브리지에서 선덜랜드와의 경기에서 5-0 승리에 기...   
16074  16085   그리고 14일 간의 사르디스 공선전후에 도시가 무너지고 리디아는 페르시아에 정복당한다.   
22252  22271  1950년 마오쩌둥의 중국과 스탈린의 소련은 호치민의 베트민을 공식적으로 인정했고,...   
4610    4612  대전 출신의 강창희 의원이 국회의장으로 사실상 확정된 상황에서 같은 대전 출신의 박...   

                                          subject_entity  \
8902   {'word': '이양수', 'start_idx': 17, 'end_idx': 19...   
30154  {'word': '첼시 FC', 'start_idx': 50, 'end_idx': ...   
16074  {'word': '리디아', 'start_idx': 31, 'end_idx': 33...   
22252  {'word': '베트민', 'start_idx': 30, 'end_idx': 32...   
4610   {'word': '박병석', 'start_idx': 45, 'end_idx': 47...   

                                           object_entity  \
8902   {'word': '자유한국당', 'start_idx': 11, 'end_idx': ...   
30154  {'word': '스탬퍼드 브리지', 'start_idx': 14, 'end_idx...   
16074  {'word': '사르디스', 'start_idx': 11, 'end

In [2]:
print(label.value_counts())
print(label.value_counts().index)

no_relation                            8563
org:top_members/employees              3851
per:employee_of                        3210
per:title                              1888
org:member_of                          1678
org:alternate_names                    1186
per:origin                             1111
org:place_of_headquarters              1075
per:date_of_birth                      1017
per:alternate_names                     898
per:spouse                              716
per:colleagues                          481
per:parents                             466
org:founded                             405
org:members                             378
per:date_of_death                       375
org:product                             342
per:children                            274
per:place_of_residence                  173
per:other_family                        171
per:place_of_birth                      149
org:founded_by                          140
per:product                     

In [3]:
replace_label = ['per:alternate_names', 'org:alternate_names', 'per:alternate_name', 'per:other_family', 'per:colleagues', 'per:siblings', 'per:spouse']
replace_member = ['org_members', 'org:menbers_of']
replace_family = ['per:parents', 'per:children']

id = len(train_dataset)
newData = []

for i in range(len(train_dataset)):
    if train_dataset.iloc[i, 4] in replace_label:
        data = train_dataset.iloc[i, :]
        
        data['id'] = id
        tmp = data['subject_entity']
        data['subject_entity'] = data['object_entity']
        data['object_entity'] = tmp

        newData.append(data)
        id += 1
    elif train_dataset.iloc[i, 4] in replace_member:
        data = train_dataset.iloc[i, :]
        
        data['id'] = id
        tmp = data['subject_entity']
        data['subject_entity'] = data['object_entity']
        data['object_entity'] = tmp

        if data['label'] == 'org_members':
            data['label'] = 'org:menbers_of'
        else: data['label'] = 'org_members'

        newData.append(data)
        id += 1
    elif train_dataset.iloc[i, 4] in replace_family:
        data = train_dataset.iloc[i, :]
        
        data['id'] = id
        tmp = data['subject_entity']
        data['subject_entity'] = data['object_entity']
        data['object_entity'] = tmp

        if data['label'] == 'per:parents': 
            data['label'] = 'per:children'
        else: data['label'] = 'per:parents'

        newData.append(data)
        id += 1


/opt/conda/lib/python3.8/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [4]:
selected_label = []
under_2000_name = [l for l in label.value_counts().index]
under_2000 = [l for l in label.value_counts()]

for i in range(len(under_2000)):
    if under_2000[i] < 2000:
        selected_label.append(under_2000_name[i])

print(selected_label)

['per:title', 'org:member_of', 'org:alternate_names', 'per:origin', 'org:place_of_headquarters', 'per:date_of_birth', 'per:alternate_names', 'per:spouse', 'per:colleagues', 'per:parents', 'org:founded', 'org:members', 'per:date_of_death', 'org:product', 'per:children', 'per:place_of_residence', 'per:other_family', 'per:place_of_birth', 'org:founded_by', 'per:product', 'per:siblings', 'org:political/religious_affiliation', 'per:religion', 'per:schools_attended', 'org:dissolved', 'org:number_of_employees/members', 'per:place_of_death']


In [5]:
import random
import re

def Random_Insertion(text, sub_word, ob_word, sub_length, ob_length):
    word_list = ['오늘', '내일', '어제', '머신', '러닝']
    new_word = word_list[random.randint(0, len(word_list) - 1)]

    text_len = len(text.split())
    text_word_list = text.split()

    sub_start = 0
    ob_start = 0

    for idx in range(len(text_word_list)):
        if re.search(sub_word , text_word_list[idx]):
            sub_start = idx
            break
    
    sub_end = sub_start + sub_length
    sub_index = [i for i in range(sub_start, sub_end)]

    for idx in range(len(text_word_list)):
        if re.search(ob_word , text_word_list[idx]):
            ob_start = idx
            break

    ob_end = ob_start + ob_length
    ob_index = [i for i in range(ob_start, ob_end)]

    valid_num = [i for i in range(0, text_len) if i not in sub_index and i not in ob_index]
    
    if valid_num != []:
        target_num = random.choice(valid_num)
        text_word_list.insert(target_num, new_word)
    else:
        target_num = random.randInt(0, text_len)
        text_word_list.insert(target_num, new_word)

    return ' '.join(text_word_list)


In [6]:
for idx in range(len(train_dataset)):
    if train_dataset.iloc[idx, 4] in selected_label:
        data = train_dataset.iloc[idx, :]

        subject_dict = {}
        subject_entity = eval(data['subject_entity'])
        subject_word = subject_entity['word']
        subject_word_list = subject_word.split()
        
        object_dict = {}
        object_entity = eval(data['object_entity'])
        object_word = object_entity['word']
        object_word_list = object_word.split()

        try:
            data['id'] = id
            data['sentence'] = Random_Insertion(data['sentence'], subject_word_list[0], object_word_list[0], len(subject_word.split()), len(object_word.split()))
            
            subject_dict = {'word': f'{subject_word}', 'start_idx': re.search(subject_word, data['sentence']).start(), 'end_idx': re.search(subject_word, data['sentence']).end()-1}
            object_dict = {'word': f'{object_word}', 'start_idx': re.search(object_word, data['sentence']).start(), 'end_idx': re.search(object_word, data['sentence']).end()-1}

            data['subject_entity'] = subject_dict
            data['object_entity'] = object_dict

            newData.append(data)
            id += 1
        except:
            continue

In [7]:
newDF = pd.DataFrame(newData, columns=['id', 'sentence', 'subject_entity', 'object_entity', 'label', 'source'])
train_dataset = train_dataset.append(newDF, ignore_index=True)

print(newDF['label'].value_counts())
print(newDF.shape)

print(train_dataset['label'].value_counts())
print(train_dataset.shape)

print(dev_dataset['label'].value_counts())
print(dev_dataset.shape)

newDF.to_csv('../dataset/train/addDataset.csv', index=False)
train_dataset.to_csv('../dataset/train/train_dataset.csv', index=False)
dev_dataset.to_csv('../dataset/train/dev_dataset.csv', index=False)

org:alternate_names                    2360
per:title                              1884
per:alternate_names                    1766
org:member_of                          1661
per:spouse                             1426
per:origin                             1106
org:place_of_headquarters              1070
per:date_of_birth                      1016
per:colleagues                          961
per:parents                             733
per:children                            732
org:founded                             403
org:members                             373
per:date_of_death                       367
per:other_family                        340
org:product                             338
per:siblings                            244
per:place_of_residence                  170
per:place_of_birth                      147
org:founded_by                          139
per:product                             123
org:political/religious_affiliation      88
per:religion                    